# Navngiven Enhedsgenkendelse (NER)

Denne notebook er fra [AI for Beginners Curriculum](http://aka.ms/ai-beginners).

I dette eksempel vil vi lære, hvordan man træner en NER-model på [Annotated Corpus for Named Entity Recognition](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus) datasættet fra Kaggle. Før du fortsætter, skal du downloade [ner_dataset.csv](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus?resource=download&select=ner_dataset.csv) filen til den aktuelle mappe.


In [62]:
import pandas as pd
from tensorflow import keras
import numpy as np

## Forberedelse af datasættet

Vi starter med at læse datasættet ind i en dataframe. Hvis du vil lære mere om at bruge Pandas, kan du besøge en [lektion om databehandling](https://github.com/microsoft/Data-Science-For-Beginners/tree/main/2-Working-With-Data/07-python) i vores [Data Science for Beginners](http://aka.ms/datascience-beginners)


In [3]:
df = pd.read_csv('ner_dataset.csv',encoding='unicode-escape')
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


Lad os få unikke tags og oprette opslagsordbøger, som vi kan bruge til at konvertere tags til klassenumre:


In [4]:
tags = df.Tag.unique()
tags

array(['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim',
       'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat'], dtype=object)

In [8]:
id2tag = dict(enumerate(tags))
tag2id = { v : k for k,v in id2tag.items() }

id2tag[0]

'O'

Nu skal vi gøre det samme med ordforråd. For nemheds skyld vil vi oprette ordforråd uden at tage ordfrekvens i betragtning; i virkeligheden vil du måske bruge Keras vectorizer og begrænse antallet af ord.


In [14]:
vocab = set(df['Word'].apply(lambda x: x.lower()))
id2word = { i+1 : v for i,v in enumerate(vocab) }
id2word[0] = '<UNK>'
vocab.add('<UNK>')
word2id = { v : k for k,v in id2word.items() }

Vi skal oprette et datasæt med sætninger til træning. Lad os gennemgå det originale datasæt og adskille alle individuelle sætninger i `X` (lister af ord) og `Y` (liste af tokens):


In [41]:
X,Y = [],[]
s,t = [],[]
for i,row in df[['Sentence #','Word','Tag']].iterrows():
    if pd.isna(row['Sentence #']):
        s.append(row['Word'])
        t.append(row['Tag'])
    else:
        if len(s)>0:
            X.append(s)
            Y.append(t)
        s,t = [row['Word']],[row['Tag']]
X.append(s)
Y.append(t)


In [93]:
def vectorize(seq):
    return [word2id[x.lower()] for x in seq]

def tagify(seq):
    return [tag2id[x] for x in seq]

Xv = list(map(vectorize,X))
Yv = list(map(tagify,Y))

Xv[0], Yv[0]

([10386,
  23515,
  4134,
  29620,
  7954,
  13583,
  21193,
  12222,
  27322,
  18258,
  5815,
  15880,
  5355,
  25242,
  31327,
  18258,
  27067,
  23515,
  26444,
  14412,
  358,
  26551,
  5011,
  30558],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0])

For enkelhedens skyld vil vi udfylde alle sætninger med 0 tokens til den maksimale længde. I virkeligheden kunne vi vælge en mere intelligent strategi og kun udfylde sekvenser inden for én minibatch.


In [51]:
X_data = keras.preprocessing.sequence.pad_sequences(Xv,padding='post')
Y_data = keras.preprocessing.sequence.pad_sequences(Yv,padding='post')

## Definition af Tokenklassifikationsnetværk

Vi vil bruge et to-lags bidirektionelt LSTM-netværk til tokenklassifikation. For at anvende en tæt klassifikator på hver af outputtene fra det sidste LSTM-lag, vil vi bruge `TimeDistributed`-konstruktionen, som replikerer det samme tætte lag til hvert af LSTM's output ved hver trin:


In [94]:
maxlen = X_data.shape[1]
vocab_size = len(vocab)
num_tags = len(tags)
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, 300, input_length=maxlen),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.TimeDistributed(keras.layers.Dense(num_tags, activation='softmax'))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 104, 300)          9545400   
                                                                 
 bidirectional_6 (Bidirectio  (None, 104, 200)         320800    
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 104, 200)         240800    
 nal)                                                            
                                                                 
 time_distributed_3 (TimeDis  (None, 104, 17)          3417      
 tributed)                                                       
                                                                 
Total params: 10,110,417
Trainable params: 10,110,417
Non-trainable params: 0
__________________________________________

Bemærk her, at vi eksplicit angiver `maxlen` for vores datasæt - hvis vi ønsker, at netværket skal kunne håndtere sekvenser med variabel længde, skal vi være lidt mere smarte, når vi definerer netværket.

Lad os nu træne modellen. For at spare tid vil vi kun træne i én epoch, men du kan prøve at træne i længere tid. Derudover kan du overveje at opdele en del af datasættet som træningsdatasæt for at observere valideringsnøjagtighed.


In [57]:
model.fit(X_data,Y_data)

1499/1499 [==============================] - 740s 488ms/step - loss: 0.0667 - acc: 0.9841


## Test af resultatet

Lad os nu se, hvordan vores entitetsgenkendelsesmodel fungerer på en eksempel-sætning:


In [91]:
sent = 'John Smith went to Paris to attend a conference in cancer development institute'
words = sent.lower().split()
v = keras.preprocessing.sequence.pad_sequences([[word2id[x] for x in words]],padding='post',maxlen=maxlen)
res = model(v)[0]

In [92]:
r = np.argmax(res.numpy(),axis=1)
for i,w in zip(r,words):
    print(f"{w} -> {id2tag[i]}")

john -> B-per
smith -> I-per
went -> O
to -> O
paris -> B-geo
to -> O
attend -> O
a -> O
conference -> O
in -> O
cancer -> B-org
development -> I-org
institute -> I-org


## Vigtig pointe

Selv en simpel LSTM-model viser rimelige resultater ved NER. Men for at opnå langt bedre resultater kan det være en fordel at bruge store forudtrænede sprogmodeller som BERT. Træning af BERT til NER ved hjælp af Huggingface Transformers-biblioteket er beskrevet [her](https://huggingface.co/course/chapter7/2?fw=pt).



---

**Ansvarsfraskrivelse**:  
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på nøjagtighed, skal du være opmærksom på, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel menneskelig oversættelse. Vi påtager os intet ansvar for misforståelser eller fejltolkninger, der måtte opstå som følge af brugen af denne oversættelse.
